# 거래량과 종가

In [34]:
import yfinance as yf

# 삼성전자 (예: 005930.KS), 기간: 최근 6개월
ticker = '005930.KS'  # '.KS'는 한국거래소(KRX) 종목
stock = yf.Ticker(ticker)

# 주가 데이터 다운로드 (시작일~종료일 지정)
df = stock.history(start="2025-01-01", end="2025-06-01")   #train
df2 = stock.history(start="2025-06-01", end="2025-06-08")   #test

In [35]:
df["Volume_lag1"]=df["Volume"].shift(1)
df["Volume_lag2"]=df["Volume"].shift(2)
df=df.dropna()
df2["Volume_lag1"]=df2["Volume"].shift(1)
df2["Volume_lag2"]=df2["Volume"].shift(2)
df2=df2.dropna()

In [37]:
#거래량->종가를 regression, 2025, 다른 기법
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
model1 = GradientBoostingRegressor(n_estimators=50, max_depth=None,
                                   learning_rate=0.01).fit(
    df[["Volume_lag1","Volume_lag2"]], df.Close)
pred = model1.predict( df2[["Volume_lag1","Volume_lag2"]])
mean_squared_error( df2.Close, pred)**0.5

3900.1106890811425

In [30]:
#Volume_lag1, Volume shift1
#Volume_lag2, shft2
#Na -> dropna
#X: Volume_lag1, Volume_lag2
#df["Volume"].shift(3)

,Volume
Date,
2025-01-02 00:00:00+09:00,NaN
2025-01-03 00:00:00+09:00,NaN
2025-01-06 00:00:00+09:00,NaN
2025-01-07 00:00:00+09:00,16630538.0
2025-01-08 00:00:00+09:00,19318046.0
...,...
2025-05-26 00:00:00+09:00,7794181.0
2025-05-27 00:00:00+09:00,15254278.0
2025-05-28 00:00:00+09:00,11247115.0


In [ ]:
# 어제 거래량 컬럼 생성 (lag)
df['Volume_lag1'] = df['Volume'].shift(1)

# NaN 제거 (첫 행은 어제 데이터 없음)
df = df.dropna()

# 결과 확인
print(df[['Close', 'Volume', 'Volume_lag1']].head())

                             Close    Volume  Volume_lag1
Date                                                     
2025-05-12 00:00:00+09:00  57600.0  15414702    7814322.0
2025-05-13 00:00:00+09:00  56900.0  16842801   15414702.0
2025-05-14 00:00:00+09:00  57400.0  12468089   16842801.0
2025-05-15 00:00:00+09:00  57300.0  13139736   12468089.0
2025-05-16 00:00:00+09:00  56800.0  10385352   13139736.0


# 범주형 변수와 교호작용

In [38]:
import pandas as pd

# 데이터 생성
data = pd.DataFrame({
    'EngineSize': [1.6, 2.0, 3.0, 1.4, 2.2, 2.5, 3.5, 1.2],
    'FuelType': ['Gasoline', 'Diesel', 'Gasoline', 'Diesel', 'Gasoline', 'Diesel', 'Gasoline', 'Diesel'],
    'Price': [18000, 22000, 27000, 16000, 24000, 25000, 32000, 15000]
})

In [44]:
# 범주형 변수 'FuelType'을 더미변수로 변환
data_encoded = pd.get_dummies(data, columns=['FuelType'], drop_first=True)

In [48]:
data_encoded['Interaction'] = data_encoded['EngineSize'] * data_encoded['FuelType_Gasoline']
print(data_encoded)

   EngineSize  Price  FuelType_Gasoline  Interaction
0         1.6  18000               True          1.6
1         2.0  22000              False          0.0
2         3.0  27000               True          3.0
3         1.4  16000              False          0.0
4         2.2  24000               True          2.2
5         2.5  25000              False          0.0
6         3.5  32000               True          3.5
7         1.2  15000              False          0.0


In [49]:
# 회귀 모델 적합
from statsmodels.formula.api import ols
model = ols(formula='Price~EngineSize+FuelType_Gasoline+Interaction', data=data_encoded).fit()

# 결과 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     70.43
Date:                Mon, 09 Jun 2025   Prob (F-statistic):           0.000643
Time:                        07:08:15   Log-Likelihood:                -64.240
No. Observations:                   8   AIC:                             136.5
Df Residuals:                       4   BIC:                             136.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=8 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [ ]:
from sklearn.linear_model import LinearRegression
# 독립변수(X)와 종속변수(y) 설정
X = data_encoded[['EngineSize', 'FuelType_Gasoline', 'Interaction']]
y = data_encoded['Price']

# 모델 학습
reg = LinearRegression()
reg.fit(X, y)

# 계수 출력
print("절편:", reg.intercept_)
print("회귀계수:", reg.coef_)

절편: 5181.384248210023
회귀계수: [ 8066.82577566  2488.41598681 -1239.563731  ]


- steel plate fault 회귀

In [61]:
df = pd.read_csv("Faults.NNA", sep="\t", header=None)
columns = ['X'+str(i) for i in range(1,28)] + ['Y'+str(i) for i in range(1,8)]
df.columns=columns

In [66]:
df["Y8"] = df.iloc[:, 27:34].sum(axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
model1 = GradientBoostingRegressor().fit( train.iloc[:, :27], train.Y8)
pred = model1.predict( test.iloc[:, :27])

from sklearn.metrics import mean_squared_error
mean_squared_error( test.Y8, pred)**0.5